In [2]:
import fmpy
#import pyfmi
fmu = 'plant.fmu'
fmpy.dump(fmu)  # get information
res = fmpy.simulate_fmu(fmu)
#model = pyfmi.load_fmu(fmu)
#model.get_model_variables()


Model Info

  FMI Version        2.0
  FMI Type           Co-Simulation
  Model Name         plant
  Description        
  Platforms          win32, win64
  Continuous States  0
  Event Indicators   1
  Variables          253
  Generation Tool    DSHplus 3.14-dev | time limited +
  Generation Date    2022-02-22T22:24:54Z

Default Experiment

  Stop Time          1.0
  Tolerance          1e-06

Variables (input, output)

  Name               Causality              Start Value  Unit     Description
  position           output                                       SignalAusgang1_Output0
  Schaltventil1      input                          0.0           SignalEingang1_Input0
  Servoventil        input                          0.0           SignalEingang1_Input1
  Schaltventil2      input                          0.0           SignalEingang1_Input2


Exception: The FMU cannot be simulated on the current platform (darwin64).